第一部分的做法是将序列直接看成普通特征，用sklearn中的各种回归试试效果，在最后还试了一个两层的神经网络，效果还是没普通的线性回归好。最终交了线性回归的预测结果，成绩很差，比直接交28好还差。

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torch.backends.cudnn as cudnn
cudnn.benchmark = True

np.random.seed(0)

### 读取验证集

In [ ]:
pred_day = 28
metro_file = pd.read_csv('./Metro_testA/testA_record_2019-01-'+str(pred_day).zfill(2)+'.csv')
metro_file.time = pd.to_datetime(metro_file.time)
start_time = '2019-01-'+str(pred_day)+' 00:00:00'
start_time = pd.to_datetime(start_time)
end_time = '2019-01-'+str(pred_day)+' 00:10:00'
end_time = pd.to_datetime(end_time)
tenmin_delta = end_time - start_time
metro_dfs = []
for k in range(81):
    metro_dfs.append(pd.DataFrame(np.zeros((144, 11))))
for j in range(144):
    metro_tmp = metro_file[(metro_file.time>=start_time) & (metro_file.time<end_time)].groupby(['stationID', 'status', 'payType']).count()
    for k in range(81):
        try:
            metro_dfs[k].loc[j, 0] = metro_tmp.loc[k, 1].time.sum()
        except:
            metro_dfs[k].loc[j, 0] = 0
        try:
            metro_dfs[k].loc[j, 1] = metro_tmp.loc[k, 1, 0].time
        except:
            metro_dfs[k].loc[j, 1] = 0
        try:
            metro_dfs[k].loc[j, 2] = metro_tmp.loc[k, 1, 1].time
        except:
            metro_dfs[k].loc[j, 2] = 0
        try:
            metro_dfs[k].loc[j, 3] = metro_tmp.loc[k, 1, 2].time
        except:
            metro_dfs[k].loc[j, 3] = 0
        try:
            metro_dfs[k].loc[j, 4] = metro_tmp.loc[k, 1, 3].time
        except:
            metro_dfs[k].loc[j, 4] = 0
        try:
            metro_dfs[k].loc[j, 5] = metro_tmp.loc[k, 0].time.sum()
        except:
            metro_dfs[k].loc[j, 5] = 0
        try:
            metro_dfs[k].loc[j, 6] = metro_tmp.loc[k, 0, 0].time
        except:
            metro_dfs[k].loc[j, 6] = 0
        try:
            metro_dfs[k].loc[j, 7] = metro_tmp.loc[k, 0, 1].time
        except:
            metro_dfs[k].loc[j, 7] = 0
        try:
            metro_dfs[k].loc[j, 8] = metro_tmp.loc[k, 0, 2].time
        except:
            metro_dfs[k].loc[j, 8] = 0
        try:
            metro_dfs[k].loc[j, 9] = metro_tmp.loc[k, 0, 3].time
        except:
            metro_dfs[k].loc[j, 9] = 0
        if start_time.weekday()<=4:
            metro_dfs[k].loc[j, 10] = 0
        else:
            metro_dfs[k].loc[j, 10] = 1
    start_time = end_time
    end_time += tenmin_delta

In [ ]:
for i in range(81):
    metro_dfs[i].rename(columns={0:'in_num', 1:'in_paytype_0', 2:'in_paytype_1', 3:'in_paytype_2', 4:'in_paytype_3', 
                                 5:'out_num', 6:'out_paytype_0', 7:'out_paytype_1', 8:'out_paytype_2', 9:'out_paytype_3',
                                 10:'weekend'}, inplace=True)

In [ ]:
pred_day = 28
eval_features = np.zeros((81, 2, 1, 144 * 6 + 2))
all_min_max = np.load('./Metro_data/all_min_max.npy').item()
for istation in range(81):
    if istation != 54:
        istation_min_max = all_min_max['station-'+str(istation).zfill(2)]
        metro_dfs[istation].iloc[:, [0,5,1,2,3,4,6,7,8,9]] = (metro_dfs[istation].iloc[:, [0,5,1,2,3,4,6,7,8,9]] - np.array([istation_min_max['in']['in_num_min'],
                                                       istation_min_max['in']['out_num_min'],
                                                       istation_min_max['in']['in_paytype_0_min'],
                                                       istation_min_max['in']['in_paytype_1_min'],
                                                       istation_min_max['in']['in_paytype_2_min'],
                                                       istation_min_max['in']['in_paytype_3_min'],
                                                       istation_min_max['out']['out_paytype_0_min'],
                                                       istation_min_max['out']['out_paytype_1_min'],
                                                       istation_min_max['out']['out_paytype_2_min'],
                                                       istation_min_max['out']['out_paytype_3_min'],
                                                      ]))/(np.array([istation_min_max['in']['in_num_max'],
                                                                    istation_min_max['in']['out_num_max'],
                                                                    istation_min_max['in']['in_paytype_0_max'],
                                                                    istation_min_max['in']['in_paytype_1_max'],
                                                                    istation_min_max['in']['in_paytype_2_max'],
                                                                    istation_min_max['in']['in_paytype_3_max'],
                                                                    istation_min_max['out']['out_paytype_0_max'],
                                                                    istation_min_max['out']['out_paytype_1_max'],
                                                                    istation_min_max['out']['out_paytype_2_max'],
                                                                    istation_min_max['out']['out_paytype_3_max'],
                                     ])-np.array([istation_min_max['in']['in_num_min'],
                                                  istation_min_max['in']['out_num_min'],
                                                  istation_min_max['in']['in_paytype_0_min'],
                                                  istation_min_max['in']['in_paytype_1_min'],
                                                  istation_min_max['in']['in_paytype_2_min'],
                                                  istation_min_max['in']['in_paytype_3_min'],
                                                  istation_min_max['out']['out_paytype_0_min'],
                                                  istation_min_max['out']['out_paytype_1_min'],
                                                  istation_min_max['out']['out_paytype_2_min'],
                                                  istation_min_max['out']['out_paytype_3_min'],
                                                 ]))
        if pd.to_datetime('2019-01-'+str(pred_day+1).zfill(2)).weekday()<=4:
            weekend = np.array((metro_dfs[0].iloc[2,-1], 0))
        else:
            weekend = np.array((metro_dfs[0].iloc[2,-1], 1))
        station_feature = metro_dfs[istation]
        for jinout in range(2):
            if jinout == 0:
                eval_features[istation, jinout, 0, :] = np.hstack((station_feature.iloc[:,[0,5,1,2,3,4]].values.reshape(-1),weekend))
            else:
                eval_features[istation, jinout, 0, :] = np.hstack((station_feature.iloc[:,[0,5,6,7,8,9]].values.reshape(-1),weekend))
np.save('eval_feature.npy', eval_features)

### 读取训练集和测试集

In [ ]:
train_feature = np.load('train_feature.npy')
train_label = np.load('train_label.npy')
test_feature = np.load('test_feature.npy')
test_label = np.load('test_label.npy')

### 浅层回归方法

In [ ]:
def fit_model(model, train_feature, train_label, test_feature, test_label):
    model.fit(train_feature, train_label)
    train_pred_label = model.predict(train_feature)
    test_pred_label = model.predict(test_feature)
    train_loss = mean_squared_error(train_pred_label, train_label)
    test_loss = mean_squared_error(test_pred_label, test_label)
    return train_loss, test_loss

In [ ]:
def fit_pred_model(model, train_feature, train_label, test_feature, test_label, eval_feature):
    model.fit(train_feature, train_label)
    train_pred_label = model.predict(train_feature)
    test_pred_label = model.predict(test_feature)
    train_loss = mean_squared_error(train_pred_label, train_label)
    test_loss = mean_squared_error(test_pred_label, test_label)
    eval_pred_label = model.predict(eval_feature)
    return eval_pred_label

注释行左边为训练总MSE，右边为测试总MSE。

In [ ]:
# from sklearn import tree
# model = tree.DecisionTreeRegressor()
# # 0.0 0.627775450234844

from sklearn import linear_model
model = linear_model.LinearRegression()
# 4.908710878115789e-29 0.30115703199174515

# from sklearn import neighbors
# model = neighbors.KNeighborsRegressor(n_neighbors=2)
# # 0.08803345179195304 0.34083946240049395

# from sklearn import ensemble
# model = ensemble.RandomForestRegressor(n_estimators=20)
# # 0.07614751868807886 0.5845056823843853

# from sklearn.ensemble import BaggingRegressor
# model = BaggingRegressor()
# # 0.08886360769056594 0.6181388945928975

# from sklearn.tree import ExtraTreeRegressor
# model = ExtraTreeRegressor()
# # 0.0 0.42660316809456345

all_train_loss = 0
all_test_loss = 0
for istation in range(81):
    for jinout in range(2):
        model = linear_model.LinearRegression()
        train_loss, test_loss = fit_model(model, train_feature[istation][jinout], train_label[istation][jinout], test_feature[istation][jinout], test_label[istation][jinout])
        all_train_loss += train_loss
        all_test_loss += test_loss
print(all_train_loss, all_test_loss)

In [ ]:
eval_label = fit_pred_model(model, train_feature[0][0], train_label[0][0], 
                            test_feature[0][0], test_label[0][0], eval_features[0][0])

### 两层神经网络回归

In [ ]:
train_feature = torch.from_numpy(train_feature)
train_feature = train_feature.float()
train_feature = Variable(train_feature)
train_feature = train_feature.cuda()

train_label = torch.from_numpy(train_label)
train_label = train_label.float()
train_label = Variable(train_label)
train_label = train_label.cuda()

test_feature = torch.from_numpy(test_feature)
test_feature = test_feature.float()
test_feature = Variable(test_feature)
test_feature = test_feature.cuda()

In [ ]:
class FCNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=144):
        super(FCNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
net = FCNet(input_size = train_feature.shape[-1], hidden_size = 144 * 3, output_size=144)
net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
net.cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

In [ ]:
# 0.07769668544656838 0.5089264005598018
train_loss = 0
test_loss = 0
for istation in range(81):
    for jinout in range(2):
        net = FCNet(input_size = train_feature.shape[-1], hidden_size = 144 * 3, output_size=144)
        net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
        net.cuda()
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
        for epoch in range(128):
            net.train()
            out = net(train_feature[istation][jinout])
            loss = criterion(out, train_label[istation][jinout])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            net.eval()
            pred_label = net(test_feature[istation][jinout])
            pred_label = pred_label.data.cpu().numpy()
            mse = np.mean((pred_label - test_label[istation][jinout]) ** 2)
        train_loss += loss.item()
        test_loss += mse
print(train_loss, test_loss)

### 预测
比较之后，最终用线性回归进行预测。

In [ ]:
result = np.zeros((11664, 2))
for istaiton in range(81):
    for jinout in range(2):
        model = linear_model.LinearRegression()
        eval_label = fit_pred_model(model, train_feature[istation][jinout], train_label[istation][jinout], test_feature[istation][jinout], test_label[istation][jinout], eval_features[istaiton][jinout])
        if jinout == 0:
            ij_min = all_min_max['station-'+str(istaiton).zfill(2)]['in']['label_min']
            ij_max = all_min_max['station-'+str(istaiton).zfill(2)]['in']['label_max']
        else:
            ij_min = all_min_max['station-'+str(istaiton).zfill(2)]['out']['label_min']
            ij_max = all_min_max['station-'+str(istaiton).zfill(2)]['out']['label_max']
        eval_label = eval_label * (ij_max - ij_min) + ij_min
        eval_label = pd.DataFrame(eval_label.transpose())
        eval_label[eval_label.iloc[:,0]<0] = 0
        eval_label = np.array(eval_label).round().reshape(-1)
        result[istaiton*144:(istaiton+1)*144, jinout] = eval_label
result = pd.DataFrame(result)
result.to_csv('pred.csv', index=False)